<a href="https://colab.research.google.com/github/guimCC/Novartis_Datathon_2023/blob/main/datathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install pandas pyarrow

   brand     phase  country  dayweek  month   wd_perc ther_area  \
0  AIMST  0.006284  Aldovia      2.0    1.0  0.045455       NaN   
1  AIMST  0.123459  Aldovia      3.0    1.0  0.090909       NaN   
2  AIMST  0.055607  Aldovia      4.0    1.0  0.136364       NaN   
3  AIMST  0.032148  Aldovia      0.0    1.0  0.181818       NaN   
4  AIMST  0.097054  Aldovia      1.0    1.0  0.227273       NaN   

   hospital_rate  n_nwd_bef  n_nwd_aft  n_weekday_0  n_weekday_1  n_weekday_2  \
0            NaN        4.0        0.0            4            4            5   
1            NaN        0.0        0.0            4            4            5   
2            NaN        0.0        2.0            4            4            5   
3            NaN        2.0        0.0            4            4            5   
4            NaN        0.0        0.0            4            4            5   

   n_weekday_3  n_weekday_4       date  wd  wd_left   monthly main_channel  
0            5            4 2013-

In [ ]:
import pandas as pd

df_train = pd.read_parquet('train_data.parquet')
df_test = pd.read_parquet('submission_data.parquet')
df_test.head()


,country,brand,dayweek,month,wd_perc,ther_area,hospital_rate,n_nwd_bef,n_nwd_aft,n_weekday_0,n_weekday_1,n_weekday_2,n_weekday_3,n_weekday_4,date,wd,wd_left,main_channel
5356,Aldovia,AIMST,0.0,1.0,0.047619,NaN,NaN,2.0,0.0,5,4,4,4,4,2022-01-03,1,20,NaN
5357,Aldovia,AIMST,1.0,1.0,0.095238,NaN,NaN,0.0,0.0,5,4,4,4,4,2022-01-04,2,19,NaN
5358,Aldovia,AIMST,2.0,1.0,0.142857,NaN,NaN,0.0,0.0,5,4,4,4,4,2022-01-05,3,18,NaN
5359,Aldovia,AIMST,3.0,1.0,0.190476,NaN,NaN,0.0,0.0,5,4,4,4,4,2022-01-06,4,17,NaN
5360,Aldovia,AIMST,4.0,1.0,0.238095,NaN,NaN,0.0,2.0,5,4,4,4,4,2022-01-07,5,16,NaN


In [ ]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.2 MB/s eta 0:00:00


In [ ]:
import category_encoders as ce
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor


# Load the data
df_train = pd.read_parquet('train_data.parquet')
df_test = pd.read_parquet('submission_data.parquet')

# Select columns of interest and convert 'date' to datetime
df_train = df_train[['country', 'brand', 'date', 'phase']]
df_train['date'] = pd.to_datetime(df_train['date'])

# https://stackoverflow.com/questions/68441559/how-to-convert-dtype-datetime64ns-to-float/71259141#71259141
def dt64_to_float(dt64):
    year = dt64.astype('M8[Y]')
    days = (dt64 - year).astype('timedelta64[D]')
    year_next = year + np.timedelta64(1, 'Y')
    days_of_year = (year_next.astype('M8[D]') - year.astype('M8[D]')).astype('timedelta64[D]')
    dt_float = 1970 + year.astype(float) + days / (days_of_year)
    return dt_float




# Initialize the TargetEncoder
encoder = ce.TargetEncoder(cols=['country', 'brand'])

# Encode 'country' and 'brand' in the entire training dataset
df_train_encoded = encoder.fit_transform(df_train[['country', 'brand']], df_train['phase'])
df_train[['country', 'brand']] = df_train_encoded[['country', 'brand']]

# Separate training data into before 2021 and for 2021
df_val_2021 = df_train[df_train['date'].dt.year == 2021]
df_train_before_2021 = df_train[df_train['date'].dt.year.isin([2020, 2019])]

df_val_2021['date'] = dt64_to_float(df_val_2021['date'].to_numpy())
df_train_before_2021['date'] = dt64_to_float(df_train_before_2021['date'].to_numpy())



# Features and target for training before 2021
X_train = df_train_before_2021[['country', 'brand', 'date']]
y_train = df_train_before_2021['phase']

# Features and target for validation in 2021
X_val = df_val_2021[['country', 'brand', 'date']]
y_val = df_val_2021['phase']

# Initialize and train a Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the validation set
y_val_pred = rf_model.predict(X_val)




from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_val, y_val_pred)
print(f'Mean Absolute Error (MAE): {mae:.2f}')

# Mean Squared Error (MSE)
mse = mean_squared_error(y_val, y_val_pred)
print(f'Mean Squared Error (MSE): {mse:.2f}')

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')

# R-squared (Coefficient of Determination)
r_squared = r2_score(y_val, y_val_pred)
print(f'R-squared (Coefficient of Determination): {r_squared:.2f}')

# Percentage of Correct Sign Predictions
correct_sign_predictions = np.sum(np.sign(y_val * y_val_pred) == 1) / len(y_val) * 100
print(f'Percentage of Correct Sign Predictions: {correct_sign_predictions:.2f}%')





<ipython-input-19-3b40e9b6c57a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['date'] = pd.to_datetime(df_train['date'])
<ipython-input-19-3b40e9b6c57a>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val_2021['date'] = dt64_to_float(df_val_2021['date'].to_numpy())
<ipython-input-19-3b40e9b6c57a>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

Mean Absolute Error (MAE): 0.05
Mean Squared Error (MSE): 0.01
Root Mean Squared Error (RMSE): 0.07
R-squared (Coefficient of Determination): -0.62
Percentage of Correct Sign Predictions: 55.20%


In [ ]:

def metric(df: pd.DataFrame) -> float:
    """Compute performance metric.

    :param df: Dataframe with target ('phase') and 'prediction', and identifiers.
    :return: Performance metric
    """
    df = df.copy()
    assert 'monthly' in df.columns, "Missing 'monthly' column, only available in the train set"
    assert 'phase' in df.columns, "Missing 'phase' column, only available in the train set"
    assert 'prediction' in df.columns, "Missing 'prediction' column with predictions"

    df["date"] = pd.to_datetime(df["date"])

    # create datetime columns
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['quarter'] = df['date'].dt.quarter

    # Sum of phasing country-brand-month = 1
    df['sum_pred'] = df.groupby(['year', 'month', 'brand', 'country'])['prediction'].transform(sum)
    #assert np.isclose(df['sum_pred'], 1.0, rtol=1e-04).all(), "Condition phasing year-month-brand-country must sum 1 is not fulfilled"

    # define quarter weights
    df['quarter_w'] = np.where(df['quarter'] == 1, 1,
                    np.where(df['quarter'] == 2, 0.75,
                    np.where(df['quarter'] == 3, 0.66, 0.5)))

    # compute and return metric
    return round(np.sqrt((1 / len(df)) * sum(((df['phase'] - df['prediction'])**2) * df['quarter_w'] * df['monthly'])), 8)

def metric(df: pd.DataFrame) -> float:
    """Compute performance metric.

    :param df: Dataframe with target ('phase') and 'prediction', and identifiers.
    :return: Performance metric
    """
    df = df.copy()
    assert 'monthly' in df.columns, "Missing 'monthly' column, only available in the train set"
    assert 'phase' in df.columns, "Missing 'phase' column, only available in the train set"
    assert 'prediction' in df.columns, "Missing 'prediction' column with predictions"

    df["date"] = pd.to_datetime(df["date"])

    # create datetime columns
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['quarter'] = df['date'].dt.quarter

    # Sum of phasing country-brand-month = 1
    df['sum_pred'] = df.groupby(['year', 'month', 'brand', 'country'])['prediction'].transform(sum)
    #assert np.isclose(df['sum_pred'], 1.0, rtol=1e-04).all(), "Condition phasing year-month-brand-country must sum 1 is not fulfilled"

    # define quarter weights
    df['quarter_w'] = np.where(df['quarter'] == 1, 1,
                    np.where(df['quarter'] == 2, 0.75,
                    np.where(df['quarter'] == 3, 0.66, 0.5)))

    # compute and return metric
    return round(np.sqrt((1 / len(df)) * sum(((df['phase'] - df['prediction'])**2) * df['quarter_w'] * df['monthly'])), 8)

df_train = pd.read_parquet('train_data.parquet')
df_train = df_train[df_train['date'].dt.year.isin([2021])]

df_val_predictions = pd.DataFrame({'phase': y_val, 'prediction': y_val_pred, 'monthly': df_train['monthly'], 'date': X_val['date'], 'brand': X_val['brand'], 'country': X_val['country']})

# Use the metric function to compute the performance metric
performance_metric = metric(df_val_predictions)


print(f'Performance Metric: {performance_metric}')





Performance Metric: 0.0172611


In [25]:
df_train = pd.read_parquet('train_data.parquet')
print(len(np.unique(df_train['brand'])))
print(len(np.unique(df_train['country'])))

258
30


In [ ]:
from sklearn.metrics import mean_squared_error
# Calculate Mean Squared Error
mse = mean_squared_error(y_val, y_val_pred)

# Print the MSE
print(f'Mean Squared Error: {mse:.2f}')

Mean Squared Error: 0.01


In [ ]:
df = df_val_predictions

df["date"] = pd.to_datetime(df["date"])

# create datetime columns
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter


df['sum_pred'] = df.groupby(['year', 'month', 'brand', 'country'])['prediction'].transform(sum)
df['sum_pred']

1789       7.874314
1790       7.874314
1791       7.874314
1792       7.874314
1793       7.874314
             ...   
3523148    0.000000
3523149    0.000000
3523150    0.000000
3523151    0.000000
3523152    0.000000
Name: sum_pred, Length: 306913, dtype: float64

In [ ]:
def metric(df: pd.DataFrame) -> float:
    """Compute performance metric.

    :param df: Dataframe with target ('phase') and 'prediction', and identifiers.
    :return: Performance metric
    """
    df = df.copy()
    assert 'monthly' in df.columns, "Missing 'monthly' column, only available in the train set"
    assert 'phase' in df.columns, "Missing 'phase' column, only available in the train set"
    assert 'prediction' in df.columns, "Missing 'prediction' column with predictions"

    df["date"] = pd.to_datetime(df["date"])

    # create datetime columns
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['quarter'] = df['date'].dt.quarter

    # Sum of phasing country-brand-month = 1
    df['sum_pred'] = df.groupby(['year', 'month', 'brand', 'country'])['prediction'].transform(sum)
    assert np.isclose(df['sum_pred'], 1.0, rtol=1e-04).all(), "Condition phasing year-month-brand-country must sum 1 is not fulfilled"

    # define quarter weights
    df['quarter_w'] = np.where(df['quarter'] == 1, 1,
                    np.where(df['quarter'] == 2, 0.75,
                    np.where(df['quarter'] == 3, 0.66, 0.5)))

    # compute and return metric
    return round(np.sqrt((1 / len(df)) * sum(((df['phase'] - df['prediction'])**2) * df['quarter_w'] * df['monthly'])), 8)



In [ ]:
nan_percentage_per_column = df.isna().mean() * 100

print(nan_percentage_per_column)

brand             0.000000
phase             0.000000
country           0.000000
dayweek           0.000000
month             0.000000
wd_perc           0.000000
ther_area        12.673645
hospital_rate    17.955406
n_nwd_bef         0.000000
n_nwd_aft         0.000000
n_weekday_0       0.000000
n_weekday_1       0.000000
n_weekday_2       0.000000
n_weekday_3       0.000000
n_weekday_4       0.000000
date              0.000000
wd                0.000000
wd_left           0.000000
monthly           0.000000
main_channel     17.955406
dtype: float64
